# OSM Math: Week 6
## Rebekah Dix

In [2]:
import numpy as np
import scipy.optimize as opt

# Exercise 6

This routine implements the steepest descent method for quadratic functions.

In [3]:
def steep_des(xinit, Q, b, c, tol, maxIter=5000):
    """
    This function implements the steepest descent method for quadratic functions.
    """
    it = 1
    fonc = tol + 1
    xk = xinit
    
    while it < maxIter and fonc > tol:
        
        DfT = Q @ xk - b
        Df = DfT.T
        
        alphak = (Df @ DfT) / (Df @ Q @ DfT)
        
        xkp1 = xk - alphak * DfT
        
        fonc = np.linalg.norm(Df)
        
        xk = xkp1
        
        it += 1
        
    return xk

In [4]:
Q = np.array([[3, 1], [1, 1]])
b = np.array([1, 6])
c = 2
xinit = np.array([0, 1])

In [5]:
steep_des(xinit, Q, b, c, 1e-5)

array([-2.4999953,  8.4999953])

#### Compare with scipy:

In [6]:
def quad(x, *args):
    Q, b, c = args
    return .5 * x.T @ Q @ x - b.T @ x + c

In [7]:
args = Q, b, c
res = opt.minimize(quad, xinit, args=args, method='nelder-mead', options={'xtol': 1e-8, 'disp': True})

Optimization terminated successfully.
         Current function value: -22.250000
         Iterations: 104
         Function evaluations: 196


In [8]:
res.x

array([-2.49999999,  8.5       ])

It matches, but floating point/rounding errors appear to be an issue in both methods (but more so in mine).

In [9]:
np.linalg.cond(Q)

5.828427124746191

# Exercise 7

This routine computes $Df$ using forward differences and a step size of $\sqrt{\text{Rerr}}_f$.

In [10]:
def compute_Df(f, x, Rerr):
    """
    This function computes the gradient of f using forward differences.
    
    Inputs:
        f : a callable function, from R^n to R
        x (vector): a point in R^n
        Rerr (scalar): an estimate for the maximum relative error of f near x
                       Note, Rerr > e_machine
    Returns:
        Df(x) (vector): an estimate to the gradient of f at x
    """
    n = len(x)
    Df = np.zeros(n, dtype=np.float64)
    h = 2 * np.sqrt(Rerr)
    for ii in range(n):
        eii = np.eye(n)[:,ii]
        # First order forward difference:
        #Df[ii] = (f(x + h * eii) - f(x)) / (h)
        # Second order forward difference:
        Df[ii] = (-3 * f(x) + 4 * f(x + h * eii) - f(x + 2 * h * eii)) / (2 * h)
    return Df

To test this function, I'll use the Rosenbrock function and compare with a scipy routine. Note, I found that my function was more accurate when I used a second order forward difference rather than a first order forward difference.

In [11]:
def rosenbrock(x):
    return 100 * (x[1] - x[0] ** 2) ** 2 + (1 - x[0]) ** 2

In [12]:
x = np.array([-2, 2])
compute_Df(rosenbrock, x, 1e-4)

array([-1605.3648,  -400.    ])

In [13]:
opt.approx_fprime(x, rosenbrock, 1e-4)

array([-1605.799908,  -399.99    ])

# Exercise 8

In [14]:
def secant(x, x1, eps, fprime):
    """
    Inputs:
        x : vector, one initial guess
        x1 : vector, another initial guess
        eps: desired level of accuracy
        fprime : callable function, derivative of f
    
    Returns:
        xk1 : vector, an approximation to a minimizer of f
        
    """
    
    MaxIter = 3000
    it = 1
    
    xkm1 = x
    xk = x1
    
    converged = False
    while not converged and it < MaxIter:
        
        update = fprime(xk) * ((xk - xkm1) / (fprime(xk) - fprime(xkm1)))
        xk1 = xk - update
                               
        if np.abs(xk1 - xk) < np.abs(xk) * eps:
            converged = True
        else: 
            converged = False
        
        xkm1 = xk
        xk = xk1
        
        it = it + 1
    
    return xk1

In [23]:
def steepest_des(f, x, eps):
    """
    This function implements the steepest descent method
    for arbitary functions.
    
    Inputs:
        f : a callable function
        x : a vector of length n; the starting point
        eps : a "small number"
        
    Returns:
        x_min : a vector which is a close approximation 
                to the local minimzer x* of f
    
    """
    
    maxiter = 30000
    it = 1
    err = 1 + eps
    
    xk = x
    
    while err > eps and it < maxiter:
        
        Dfxk = compute_Df(f, xk, eps)
        f_alpha_prime = lambda x: compute_Df(f, xk - x * Dfxk.T, eps) @ (-1 * Dfxk.T)
        opt_alpha = secant(.8, .2, eps, f_alpha_prime)
        xkp1 = xk - opt_alpha * Dfxk
        err = np.linalg.norm(Dfxk)
        xk = xkp1
        it += 1
    
    return xk, err

# Exercise 9

In [24]:
res = steepest_des(rosenbrock, x, 1e-5)
res

(array([1.01634609, 1.03295939]), 9.999160234476975e-06)

In [25]:
rosenbrock(res[0])

0.00026719471264171474

In [21]:
compare = opt.minimize(rosenbrock, x, method='BFGS')
compare.x

array([0.99999575, 0.99999152])

In [22]:
rosenbrock(compare.x)

1.809703286310819e-11

My function approximately reaches the minimum at $(1,1)$ with value 0.